In [8]:
import re 
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import pymorphy2


def greetings(line):
    res = re.findall(r'привет|добрый день|добрый вечер|здравствуйте|приветствую|добрый|доброго', line)
    if res:
        return 'greeting=True'
    else:
        return 'greeting=False'

    
def farewell(line):
    res = re.findall(r'до свидания|пока|хорошего вечера|хорошего дня|доброй ночи|доброго дня|счастливо|до скорого|до встречи', line)
    if res:
        return 'farewell=True'
    else:
        return 'farewell=False'

    
def rules(tags, id_dig):
    global man_info 
    b = tags
    if id_dig in man_info:
        if 'greeting=True' in tags: 
            man_info[id_dig]['greeting'] = True
        if 'farewell=True' in tags:
            man_info[id_dig]['farewell'] = True
            
    else:
        man_info[id_dig] = {'greeting' : False, 'farewell' : False}
        if 'greeting=True' in tags: 
            man_info[id_dig]['greeting'] = True
        if 'farewell=True' in tags:
            man_info[id_dig]['farewell'] = True
    return b


def tokenize(text):
    words = word_tokenize(text)
    return words


def introduction(line, names):
    if ('меня' in line and 'зовут' in line) or ('это' in line):
        for i in line:
            if i in names:
                return i
    return False



def company(line):
    morph = pymorphy2.MorphAnalyzer()
    if 'компания' in line:
        index = line.index('компания')
        res = morph.parse(line[index+1])[0]
        if 'NOUN' in res.tag or 'Abbr' in res.tag:
            return res.normal_form
        if index != 0:
            res = morph.parse(line[index-1])[0]
            if 'ADJF' in res.tag or 'Abbr' in res.tag:
                return res.normal_form
    return False 



data = pd.read_csv('test_data.csv')
data['text'] = data['text'].str.lower()
data_manager = data.loc[data['role'] == 'manager']

names = pd.read_csv('name.csv')
names = names['Name'].str.lower().tolist()



    
data_manager['insight'] = data_manager.apply(lambda x: greetings(x['text']), axis = 1)
data_manager['insight'] = data_manager['insight'] + ', ' + data_manager.apply(lambda x: farewell(x['text']), axis = 1)
man_info = {}
data_manager.apply(lambda x: rules(x['insight'], x['dlg_id']), axis = 1)

for key in man_info:
    if man_info[key]['greeting']==True and man_info[key]['farewell']==True:
        print(f'В {key+1} (ID: {key}) диалоге требование выполнено')
    else:
        print(f'В {key+1} (ID: {key}) диалоге требование не выполнено')
        
        
data_manager['tokenize'] = data_manager.apply(lambda row: tokenize(row['text']), axis=1)
data_manager['name'] = data_manager.apply(lambda x: introduction(x['tokenize'], names), axis=1)
data_manager['company'] = data_manager.apply(lambda x: company(x['tokenize']), axis=1)

<ipython-input-8-b04daadd7d67>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_manager['insight'] = data_manager.apply(lambda x: greetings(x['text']), axis = 1)
<ipython-input-8-b04daadd7d67>:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_manager['insight'] = data_manager['insight'] + ', ' + data_manager.apply(lambda x: farewell(x['text']), axis = 1)
<ipython-input-8-b04daadd7d67>:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

В 1 (ID: 0) диалоге требование выполнено
В 2 (ID: 1) диалоге требование выполнено
В 3 (ID: 2) диалоге требование не выполнено
В 4 (ID: 3) диалоге требование не выполнено
В 5 (ID: 4) диалоге требование не выполнено
В 6 (ID: 5) диалоге требование не выполнено


<ipython-input-8-b04daadd7d67>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_manager['company'] = data_manager.apply(lambda x: company(x['tokenize']), axis=1)


In [9]:
data_manager.loc[data_manager['name'] != False]

,dlg_id,line_n,role,text,insight,tokenize,name,company
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...,"greeting=False, farewell=False","[меня, зовут, ангелина, компания, диджитал, би...",ангелина,диджитал
111,1,2,manager,меня зовут ангелина компания диджитал бизнес з...,"greeting=False, farewell=False","[меня, зовут, ангелина, компания, диджитал, би...",ангелина,диджитал
167,2,3,manager,меня зовут ангелина компания диджитал бизнес з...,"greeting=False, farewell=False","[меня, зовут, ангелина, компания, диджитал, би...",ангелина,диджитал
251,3,2,manager,добрый меня максим зовут компания китобизнес у...,"greeting=True, farewell=False","[добрый, меня, максим, зовут, компания, китоби...",максим,китобизнес
338,5,1,manager,да это анастасия,"greeting=False, farewell=False","[да, это, анастасия]",анастасия,False


In [10]:
data_manager.loc[data_manager['company'] != False]

,dlg_id,line_n,role,text,insight,tokenize,name,company
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...,"greeting=False, farewell=False","[меня, зовут, ангелина, компания, диджитал, би...",ангелина,диджитал
111,1,2,manager,меня зовут ангелина компания диджитал бизнес з...,"greeting=False, farewell=False","[меня, зовут, ангелина, компания, диджитал, би...",ангелина,диджитал
167,2,3,manager,меня зовут ангелина компания диджитал бизнес з...,"greeting=False, farewell=False","[меня, зовут, ангелина, компания, диджитал, би...",ангелина,диджитал
251,3,2,manager,добрый меня максим зовут компания китобизнес у...,"greeting=True, farewell=False","[добрый, меня, максим, зовут, компания, китоби...",максим,китобизнес
273,3,24,manager,транспортная компания хорошо а у вас уже вот в...,"greeting=False, farewell=False","[транспортная, компания, хорошо, а, у, вас, уж...",False,транспортный
